In [ ]:
from agential.cog.prompts.agents.react import (
    REACT_INSTRUCTION_HUMANEVAL, 
    REACT_INSTRUCTION_MBPP, 
    REACT_INSTRUCTION_HOTPOTQA
)
from agential.cog.prompts.benchmarks.humaneval import HUMANEVAL_FEWSHOT_EXAMPLES_POT
from agential.cog.prompts.benchmarks.mbpp import MBPP_FEWSHOT_EXAMPLES_REACT
from agential.cog.prompts.benchmarks.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_REACT

from agential.cog.strategies.react.code import ReActCodeStrategy
from agential.cog.agent.react import ReActAgent
import warnings
warnings.filterwarnings('ignore')

import dotenv

dotenv.load_dotenv()

from langchain_community.chat_models.openai import ChatOpenAI

llm = ChatOpenAI()

strat = ReActCodeStrategy(llm)

# QA

### HotpotQA

In [ ]:
question = 'Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring'

agent = ReActAgent(
    llm=llm,
    mode={"qa": "hotpotqa"}
)

### FEVER

### AmbigNQ

### TriviaQA

# Math

### GSM8K

### SVAMP

### TabMWP

# Code

### HumanEval

In [ ]:
inst = {"task_id": "HumanEval/0", "prompt": "from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    \"\"\" Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    \"\"\"\n", "entry_point": "has_close_elements", "canonical_solution": "    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n", "test": "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\n"}
question = inst['prompt']
tests = f"{inst['test']}\ncheck({inst['entry_point']})"

agent = ReActAgent(
    llm=llm,
    mode={"code": "humaneval"}
)

out = agent.generate(
    question=question,
    examples=HUMANEVAL_FEWSHOT_EXAMPLES_POT,
    prompt=REACT_INSTRUCTION_HUMANEVAL,
    additional_keys={},
    reset=True,
    # kwargs.
    max_steps=3
)

In [ ]:
out

### MBPP

In [ ]:
question = "Write a python function to find the first repeated character in a given string."
tests = """assert first_repeated_char("abcabc") == "a"
assert first_repeated_char("abc") == None
assert first_repeated_char("123123") == "1\""""

agent = ReActAgent(
    llm=llm,
    mode={"code": "mbpp"}
)

out = agent.generate(
    question=question,
    examples=MBPP_FEWSHOT_EXAMPLES_REACT,
    prompt=REACT_INSTRUCTION_MBPP,
    additional_keys={"tests": tests},
    reset=True,
    # kwargs.
    max_steps=3
)

In [ ]:
out